<a href="https://colab.research.google.com/github/GustavoMillerOS/lista_compras_python/blob/main/Analise_de_Delivery_de_Refei%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importações

In [ ]:
import pandas as pd
import numpy as np

# 2. Carregamento dos Dados

In [ ]:
df_refeicoes = pd.read_csv("/content/pedidos.csv")

# 3. EDA

In [ ]:
df_refeicoes.head()

In [ ]:
df_refeicoes.tail()

In [ ]:
df_refeicoes.info()


In [ ]:
df_refeicoes.describe()

# 4. Feature Engineering

In [ ]:
#Criar nova coluna Receita_Item
df_refeicoes["Receita_Item"] = df_refeicoes['Quantidade'] * df_refeicoes['Preco_Unitario']

In [ ]:
#Imprimindo a coluna Receita_Item
print(df_refeicoes[['Quantidade', 'Preco_Unitario', 'Receita_Item']].head())

In [ ]:
#Identificar valores nulos por Coluna
nulos = df_refeicoes.isnull().sum()
print(nulos)

In [ ]:
#Calcula a média da Quantidade
media_quantidade = df_refeicoes['Quantidade'].mean()

# Preencher os valores nulos com a média
df_refeicoes['Quantidade'] = df_refeicoes['Quantidade'].fillna(media_quantidade)

In [ ]:
#Remover linhas com Preco_Unitario nulo
df_refeicoes.dropna(subset=['Preco_Unitario'], inplace=True)

In [ ]:
df_refeicoes.describe()

In [ ]:
#Agrupar Item
agrupar_item = df_refeicoes.groupby('Item').agg(
    Quantidade_Total=('Quantidade', 'sum'),
    Receita_Total=('Receita_Item', 'sum')
).reset_index()

top5_itens_mais_vendidos_quantidade = agrupar_item.sort_values(by='Quantidade_Total', ascending=False).head()
top5_itens_mais_receita = agrupar_item.sort_values(by='Receita_Total', ascending=False).head()

print("Itens Agrupado:")
print(agrupar_item)
print("\nTop 5 por Quantidade:")
print(top5_itens_mais_vendidos_quantidade)
print("\nTop 5 por Receita:")
print(top5_itens_mais_receita)

# 5. Tratamento de Dados

In [ ]:
#Converter coluna Data para tipo datetime
df_refeicoes['Data'] = pd.to_datetime(df_refeicoes['Data'])

In [ ]:
df_refeicoes.info()

In [ ]:
#Analisar as Vendas por mês
df_refeicoes['Mes_venda'] = df_refeicoes['Data'].dt.month

In [ ]:
df_refeicoes.head()

In [ ]:
#Calcular receita total por mês
vendas_por_mes = df_refeicoes.groupby('Mes_venda')['Receita_Item'].sum().reset_index()
print("\nVendas por mês")
print(vendas_por_mes)

In [ ]:
# Criando uma coluna de Ano-Mês para evolução contínua
df_refeicoes['Ano_Mes'] = df_refeicoes['Data'].dt.to_period('M')

evolucao_vendas = df_refeicoes.groupby('Ano_Mes')['Receita_Item'].sum().reset_index()

evolucao_vendas['Ano_Mes'] = evolucao_vendas['Ano_Mes'].astype(str)

print("\nEvolução das Vendas (Mês a Mês):")
print(evolucao_vendas)

# 6. Agregações


In [ ]:
df_cardapio = pd.read_csv("/content/cardapio.csv")

In [ ]:
#Combinar os dados de pedidos e cardádpio
df_merge = pd.merge(df_refeicoes, df_cardapio, on='Item', how='left')
print(df_merge)

In [ ]:
#Calcular receita total por Categoria
receita_total_categoria = df_merge.groupby('Categoria')['Receita_Item'].sum().reset_index()
# Ordenando para ver qual categoria gera mais receita mais
receita_total_categoria = receita_total_categoria.sort_values(by='Receita_Item', ascending=False)
print("\nReceita Total por Categoria")
print(receita_total_categoria )

# 7. Análise Temporal

In [ ]:
#Filtre pedidos da categoria 'Salgados' com quantidade superior a 10 unidades
df_salgado = df_merge.query('Categoria == "Salgados" and Quantidade >10')
receita_por_categoria = df_merge.groupby('Categoria')['Receita_Item'].sum().reset_index()
print("Pedidos de Salgados com mais de 10 unidades:")
print(df_salgado.head())

# 8. KPIs

In [ ]:
#Recalculo preventivo NaNs anteriores
df_merge['Receita_Item'] = df_merge['Quantidade'] * df_merge['Preco_Unitario']

#Receita Total
receita_total = np.sum(df_merge['Receita_Item'])

#Total de Itens Vendidos
total_itens = np.sum(df_merge['Quantidade'])

#Ticket Médio

ticket_medio = receita_total / df_merge['ID_Pedido'].nunique()

print(f"KPIs do Negócio")
print(f"Receita Total: R$ {receita_total:,.2f}")
print(f"Total de Itens Vendidos: {total_itens:,.2f}")
print(f"Ticket Médio por Pedido: R$ {ticket_medio:,.2f}")

In [ ]:
#Calculando percentis para Preco_Unitario
p_preco = np.percentile(df_merge['Preco_Unitario'], [25, 50, 75])

#Calculando percentis para Quantidade
p_qtd = np.percentile(df_merge['Quantidade'], [25, 50, 75])

print("\nAnálise Estatística (Percentis)")
print(f"Preço Unitário - 25%: R${p_preco[0]:.2f} | 50% (Mediana): R${p_preco[1]:.2f} | 75%: R${p_preco[2]:.2f}")
print(f"Quantidade     - 25%: {p_qtd[0]:.2f} | 50% (Mediana): {p_qtd[1]:.2f} | 75%: {p_qtd[2]:.2f}")